# Conversión de imagenes en escala de grises a color usando TensorFlow

In [1]:
%pip install tensorflow-metal
%pip install matplotlib
%pip install numpy

ERROR: Could not find a version that satisfies the requirement tensorflow-metal (from versions: none)
ERROR: No matching distribution found for tensorflow-metal
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape, MaxPool2D, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split

In [3]:
print("Dispositivos físicos disponibles:", tf.config.list_physical_devices())

Dispositivos físicos disponibles: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [4]:
# Cargar conjuntos de imágenes (no es necesario aplicar transformaciones)
COLOR_PATH = 'color_processed/'
GRAY_PATH = 'gray_processed/'

def load_image_pair(): 
    gray_images = []
    color_images = []
    for img in os.listdir(GRAY_PATH): #Mismo número de imágenes en ambas carpetas
        image_gray = tf.io.read_file(os.path.join(GRAY_PATH + img)) #Cargar cada imagen
        image_color = tf.io.read_file(os.path.join(COLOR_PATH + img))
        
        gray_img_tensor_int = tf.image.decode_jpeg(image_gray, channels=1)
        color_img_tensor_int = tf.image.decode_jpeg(image_color, channels=3)
    
        gray_img_float = tf.image.convert_image_dtype(gray_img_tensor_int, tf.float32, )
        color_img_float = tf.image.convert_image_dtype(color_img_tensor_int, tf.float32)    
        
        gray_images.append(gray_img_float)
        color_images.append(color_img_float)

    return gray_images, color_images

In [ ]:
X, y = load_image_pair()
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Luego separar train de validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

In [6]:
IMAGE_SIZE = (64, 64) 
BATCH_SIZE = 128
EPOCHS = 200
LATENT_DIM = 128

In [ ]:
# Creación del Modelo Autoencoder
encoder = Sequential([
    Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),

    Conv2D(32, (3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(64, (3,3), strides=2, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(128, (3,3), strides=2, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(256, (3,3), strides=2, padding='same', activation='relu'),
    BatchNormalization(),

    Flatten(),
    Dense(LATENT_DIM, activation='relu'),  # latent_dim
    
    Dropout(0.5)
])

In [ ]:
decoder = Sequential([
    Input(shape=((LATENT_DIM,))),
    Dense(8*8*256, activation='relu'),
    Reshape((8, 8, 256)),

    Conv2DTranspose(128, (3,3), strides=2, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2DTranspose(64, (3,3), strides=2, padding='same', activation='relu'), 
    BatchNormalization(),

    Conv2DTranspose(32, (3,3), strides=2, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(3, (3,3), padding='same', activation='sigmoid')  # salida [0,1]
])

In [9]:
model = Sequential([encoder, decoder])
model.compile(optimizer='adam', loss='mae')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 128)            │     2,487,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 64, 64, 3)      │     2,502,595 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,989,635 (19.03 MB)

 Trainable params: 4,988,227 (19.03 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [10]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_autoencoder_model.keras', save_best_only=True, monitor='val_loss'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, min_delta=1e-3)    
]

In [ ]:
model.fit(np.array(X_train), np.array(y_train), 
          validation_data=(np.array(X_val), np.array(y_val)),
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          callbacks=callbacks
)

Epoch 1/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 26s 526ms/step - loss: 0.2002 - val_loss: 0.2042 - learning_rate: 0.0010
Epoch 2/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 512ms/step - loss: 0.1543 - val_loss: 0.2164 - learning_rate: 0.0010
Epoch 3/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 516ms/step - loss: 0.1480 - val_loss: 0.2125 - learning_rate: 0.0010
Epoch 4/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 515ms/step - loss: 0.1448 - val_loss: 0.2257 - learning_rate: 0.0010
Epoch 5/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 515ms/step - loss: 0.1426 - val_loss: 0.2164 - learning_rate: 5.0000e-04
Epoch 6/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 514ms/step - loss: 0.1408 - val_loss: 0.2106 - learning_rate: 5.0000e-04
Epoch 7/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 514ms/step - loss: 0.1400 - val_loss: 0.2098 - learning_rate: 5.0000e-04
Epoch 8/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 518ms/step - loss: 0.1390 - val_loss: 0.1955 - learning_rate: 2.5000e-04
Epoch 9/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 517ms/step - loss: 0.1382 - val_loss: 0.1855 